In [9]:
import numpy as np
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from statistics import mean

from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession( #rate limiter which bypasses yahoo's blocker
    limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# using now() to get current time

today = datetime.today()

# Calculate the date 21 days before today
past = today - timedelta(days=21)    

# Set the start and end date
start_date = past
end_date = today

# Define the ticker list
tickers_list = ['AAPL', 'IBM', 'MSFT', 'WMT']

# Create placeholder for data
data = pd.DataFrame(columns=tickers_list)

# Fetch the data
for ticker in tickers_list:
    data[ticker] = yf.download(ticker, 
                               start_date,
                               end_date,
                               session=session)['Adj Close']

#Adj Close, Low
    
# Print first 5 rows of the data
data.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,AAPL,IBM,MSFT,WMT
Date,,,,
2024-07-24,218.287323,182.424484,428.899994,70.599998
2024-07-25,217.238556,190.315460,418.399994,70.019997
2024-07-26,217.708008,190.087448,425.269989,69.779999
2024-07-29,217.987686,189.839615,426.730011,69.620003
2024-07-30,218.547043,189.383606,422.920013,69.190002


In [10]:
# NOTE: This next section is just for sanity checks. You can skip down to "FULL RUN" if you have faith in the system. 

prices_array = data['AAPL'].values
prices_length = len(prices_array)

print(f"{prices_array}")
print(f"{prices_length}")

[218.287323   217.23855591 217.70800781 217.98768616 218.54704285
 221.82324219 218.10754395 219.60580444 209.02806091 206.99040222
 209.5774231  213.06338501 215.99000549 217.52999878 221.27000427]
15


In [11]:
#from here:
#calculate daily returns -- might even just be able to get returns from yfinance straight
#do kelly weighting
#rank the stocks in order by their kelly weights
#can input portfolio value to know how to separate number of shares?

# 1) Calculate daily stock returns (percentage change)
daily_returns = np.diff(prices_array) / prices_array[:-1] * 100

# 2) Separate the returns into positive and negative arrays
positive_returns = daily_returns[daily_returns > 0]
negative_returns = daily_returns[daily_returns < 0]

# Output the results
print("Daily Returns:", daily_returns)
print("Positive Returns:", positive_returns)
print("Negative Returns:", negative_returns)

Daily Returns: [-0.48045259  0.21609972  0.12846489  0.25660013  1.49908198 -1.67507165
  0.68693658 -4.81669579 -0.97482543  1.24982649  1.66332893  1.37359147
  0.71299285  1.71930562]
Positive Returns: [0.21609972 0.12846489 0.25660013 1.49908198 0.68693658 1.24982649
 1.66332893 1.37359147 0.71299285 1.71930562]
Negative Returns: [-0.48045259 -1.67507165 -4.81669579 -0.97482543]


In [14]:
#Grab average, count, prob

pos_freq = len(positive_returns)

neg_freq = len(negative_returns)

pos_prob = pos_freq / (pos_freq + neg_freq)

neg_prob = neg_freq / (pos_freq + neg_freq)

pos_avg = abs(mean(positive_returns))

neg_avg = abs(mean(negative_returns))

print(pos_freq)
print(neg_freq)
print(pos_prob)
print(neg_prob)
print(neg_avg)
print(pos_avg)

10
4
0.7142857142857143
0.2857142857142857
1.9867613630361394
0.9506228649802975


In [17]:
#calculate the kelly betting weight

kelly_weight = ((pos_prob / neg_avg) - (neg_prob / pos_avg)) * 100

print(kelly_weight)

5.896783035088299


In [24]:
#FULL RUN

kelly = {}

for ticker in tickers_list:
    prices_array = data[ticker].values
    prices_length = len(prices_array)

    daily_returns = np.diff(prices_array) / prices_array[:-1] * 100

    positive_returns = daily_returns[daily_returns > 0]
    negative_returns = daily_returns[daily_returns < 0]

    pos_freq = len(positive_returns)

    neg_freq = len(negative_returns)

    pos_prob = pos_freq / (pos_freq + neg_freq)

    neg_prob = neg_freq / (pos_freq + neg_freq)

    pos_avg = abs(mean(positive_returns))

    neg_avg = abs(mean(negative_returns))

    kelly_weight = ((pos_prob / neg_avg) - (neg_prob / pos_avg)) * 100

    if kelly_weight < 0:
        kelly[ticker] = 0
    else: 
        kelly[ticker] = float(kelly_weight)

print(kelly)
    


{'AAPL': 5.896783035088299, 'IBM': 22.963172459726245, 'MSFT': 0, 'WMT': 0}


In [28]:
#and finally, portfolio weights

portfolio = 100000 #change this with portfolio size

price = {}

for ticker in tickers_list:
    prices_array = data[ticker].values
    price[ticker] = int(prices_array[-1])

print(price)

true_weights = {}
sum = 0

for ticker in tickers_list:
    sum += kelly[ticker]

print(sum)

for ticker in tickers_list:
    true_weights[ticker] = kelly[ticker] / sum

print(true_weights)

shares = {}

for ticker in tickers_list:
    shares[ticker] = int((portfolio * true_weights[ticker]) / price[ticker])

print(shares)

{'AAPL': 221, 'IBM': 190, 'MSFT': 414, 'WMT': 68}
28.859955494814542
{'AAPL': 0.20432405157893652, 'IBM': 0.7956759484210635, 'MSFT': 0.0, 'WMT': 0.0}
{'AAPL': 92, 'IBM': 418, 'MSFT': 0, 'WMT': 0}
